

---
# Deep Learning With Tensorflow for Computer vision

Par **Mr Abdoulaye Balde** <br>
[Github](https://github.com/abdoulayegk/ODC-Formaton )  <br>
[Twitter](https://twitter.com/abdoulayegk)




---
L’ensemble de données que nous allons utiliser est très connu par les data scientists, il s’agit du Dataset MNIST, ce dernier rassemble près de 60000 images de 28×28 pixels de nombres écrits à la main avec les nombres en numérique comme variable cible. L’objectif de notre modèle sera de reconnaître le nombre écrit sur l’image.
---



In [ ]:
# Importer les librairies nécessaires
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential

# Importer les données
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape

Ici nous allons jeter un coup d’œil sur un échantillons aléatoire d’images avec leurs labels correspondant.  



In [ ]:
j = 1

for i in np.random.choice(np.arange(0, len(y_train)), size = 6):
    # Image
    img = X_train[i]

    # Subplot nous permet de selectionner une des sous-figures parmi une grille. Dans notre cas la grille
    # a 2 lignes et 3 colonnes.
    plt.subplot(2, 3, j)
    j = j + 1

    # Suppresion des axes
    plt.axis('off')

    # Affichage de la figure en niveaux de gris
    plt.imshow(img, cmap='gray', interpolation='None')

    # Modification du titre de la figure
    plt.title('Label: ' + str(y_train[i]))

# Prétraitement des données
Après le chargement de vos données, vous devez vous assurer qu’elles soient prêtes pour la phase d’entraînement. Pour cela vous appliquerez quelques transformations à ces dernières. Notamment redimensionner, normaliser et encoder les données.

Mais pas de panique, encore une fois nous vous accompagnerons dans le processus et chaque prétraitement sera expliqué.

Redimensionnement : Une particularité des algorithmes de machine learning et deep learning est qu’ils ne prennent pas de matrices en entrée. Ce qui vous amène à transformer vos matrices de 28×28 en vecteurs de taille 784.
* Normalisation : Cette étape n’est pas obligatoire pour l’entraînement de votre modèle mais peut booster sa performance, la normalisation est appliquée en divisant chaque pixel par 255.  
* Encodage : La transformation des labels en vecteurs catégoriels permet d’améliorer la précision de prédiction.

In [ ]:
# Redimensionnement
X_train = X_train.reshape((-1,28*28))
X_test = X_test.reshape((-1,28*28))

# Normalisation
X_train = X_train / 255
X_test = X_test / 255

# One hot encodding (tranformation en vecteurs categoriels)
from tensorflow.keras.utils import to_categorical

nb_classes = len(np.unique(y_train)) # Compte le nombre de classes distinctes
y_train = to_categorical(y_train, dtype="int", num_classes = nb_classes) # à exécuter qu'une seule fois
y_test = to_categorical(y_test, dtype="int", num_classes = nb_classes) # à exécuter qu'une seule fois

## Définition du modèle :
Maintenant que vos données sont (vraiment) prêtes, vous pouvez passer à la modélisation de votre réseau de neurones. C’est là que vous allez construire votre modèle couche par couche.



In [ ]:
nb_pixel = X_train.shape[1] # Nombre de pixels d'un vecteur (d'une image)

# Instanciation de la couche d'entrée
input_layer = Input(shape=nb_pixel)
# Instanciation de la première couche cachée
first_hidden = Dense(units=20, activation="relu")
# Ajout d'une deuxième couche cachée
second_hidden = Dense(units=14, activation="relu")
# Ajout de la couche de sortie
output_layer = Dense(units=nb_classes, activation="softmax")

# Construction du model
model = Sequential([
                    input_layer,
                    first_hidden,
                    second_hidden,
                    output_layer
])

## Compilation du modèle :
La méthode compile configure le processus d’entraînement du modèle en spécifiant 3 paramètres importants.

* loss : paramètre qui indique au modèle sur quelle fonction de perte se baser pour calculer l’erreur et l’optimiser. Ici nous utiliserons la « categorical_crossentropy »
* optimizer : ce paramètre définit l’algorithme d’optimisation que nous allons utiliser pour faire la descente de gradient de la fonction perte. On choisit l’optimiseur « adam » qui donne en générale de bons résultats sur un grand ensemble de problème
* metrics : paramètre servant à choisir les métriques d’évaluation du modèle pendant le processus d’entraînement. La métrique spécifiée pour ce modèle est l’accuracy (précisions), la plus utilisée pour les problèmes de classification.
L’exécution de la cellule suivante compilera votre modèle et le rendra enfin prêt pour être entraîné.

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

## Entrainement du modèle

Il y'a des parameters a passer comme arguments:
* batch_size : le nombre d’échantillons de données que contiendra un batch
* epoch : le nombre d’epochs nécessaire à l’entraînement du modèle
* validation_split : le pourcentage des données qui serviront pour l’évaluation de notre modèle pendant son entraînement.

In [ ]:
history = model.fit(X_train, y_train, batch_size=200, epochs=20, validation_split=0.2)

## Evaluation du modèle

Pour afficher la précision d’entraînement et de validation en fonction des epochs.

In [ ]:
# Afichage du graphe de precision d'apprentissage
plt.plot(range(1,20+1),
         history.history["accuracy"],
         label="Training Accuracy",
         color="red")

# Afichage du graphe de precision de vaidation
plt.plot(range(1,20+1),
         history.history["val_accuracy"],
         label="Validation Accuracy",
         color="blue")

plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

# Afficher l'accuracy et la loss finale
score = model.evaluate(X_test,y_test)
print("Fonction Perte : {:.2f}".format(score[0]))
print("Precision : {:.2f}".format(score[1]))